$$
\begin{equation}
\nabla^2 p = -2\left(\frac{\pi}{2}\right)^2\sin\left( \frac{\pi x}{L_x} \right) \cos\left(\frac{\pi y}{L_y}\right)
\end{equation}
$$

in the domain
$$
\left\lbrace \begin{align*}
0 &\leq x\leq 1  \\
-0.5 &\leq y \leq 0.5 
\end{align*} \right.
$$
where Lx=Ly=1 and with boundary conditions
$$
p=0 \text{ at } \left\lbrace 
\begin{align*}
x&=0\\
y&=0\\
y&=-0.5\\
y&=0.5
\end{align*} \right.
$$

Discretizing:
$$
\frac{p_{i+1,j}^{k}-2p_{i,j}^{k}+p_{i-1,j}^{k}}{\Delta x^2}+\frac{p_{i,j+1}^{k}-2 p_{i,j}^{k}+p_{i,j-1}^{k}}{\Delta y^2}=b_{i,j}^{k}
$$

In [1]:
import numpy
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [2]:
# parameters
nx = 101
ny = 101
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

# grid
x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

# source term
b = (-2.0 * (numpy.pi / Lx) * (numpy.pi / Ly) *
     numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly))

# ICs
p0 = numpy.zeros((ny, nx))

# analytical solutions
p_exact = poisson_solution(x, y, Lx, Ly)

# Deepest Descent
The solution process:
* Calculate the residual, $r^k$, which also serves as the direction vector, $d^k$
* Calculate the step size $\alpha$
* Update $p^{k+1}=p^k+\alpha d^k$

## Residual Definition
$$
\begin{equation}
{\bf r}^k={\bf b}-A{\bf p}^k
\end{equation}
$$

In [3]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    """
    Solves the 2D Poisson equation on a uniform grid, with the same grid 
    spacing in both directions, for a given forcing term using the method 
    of steepest descent.
    
    The function assumes Dirichlet boundary conditions with value zero.
    The exit criterion of the solver is based on the relative L2-norm of 
    the solution difference between two consecutive iterations.

    Parameters
    ----------
    p0 : numpy.ndarray
        The initial solution as a 2D array of floats
    b : numpy.ndarray
        The forcing term as a 2D array of floats
    dx : float
        Grid spacing in the x direction
    dy : float
        Grid spacing in the y direction
    maxiter : integer, optional
        Maximum number of iterations to perform, default: 20000
    rtol : float, optional
        Relative tolerance for convergence, default: 1e-6

    Returns
    -------
    p : numpy.ndarray
        The solution after relaxation as a 2D array of floats
    ite : integer
        The number of iterations performed
    conv : list
        The convergence history as a list of floats
    """
    
    def A(p):
        # apply the Laplacian operator to p
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2

    p = p0.copy()
    r = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        
        # compute the residual
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        
        # compute the Laplacian of the residual
        Ar[1:-1, 1:-1] = A(r)
        
        # compute the step size
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar)
        
        # update the solution
        p = pk + alpha * r
        
        # relative L2-norm of the difference
        diff = l2_norm(p, pk)
        conv.append(diff)
        
        ite += 1
        
    return p, ite, conv

In [4]:
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy,
                                               maxiter=20000, rtol=1e-10)
print('Method of steepest descent: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_sd[-1]))

Method of steepest descent: 2 iterations to reach a relative difference of 1.3307695446303778e-16


In [5]:
l2_norm(p, p_exact)

8.225076220929745e-05

# Conjugate Gradients

Steps:
* Calculate $\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf d}^k \cdot {\bf d}^k}$
* Update $p^{k+1}$
* Calculate ${\bf r}^{k+1} = {\bf r}^k - \alpha A {\bf d}^k$
* Calculate $\beta = \frac{{\bf r}^{k+1} \cdot {\bf r}^{k+1}}{{\bf r}^k \cdot {\bf r}^k}$
* Calculate ${\bf d}^{k+1}={\bf r}^{k+1}+\beta{\bf d}^{k}$
* Repeat

In [6]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    """
    Solves the 2D Poisson equation on a uniform grid, with the same grid 
    spacing in both directions, for a given forcing term using the method 
    of conjugate gradients.
    
    The function assumes Dirichlet boundary conditions with value zero.
    The exit criterion of the solver is based on the relative L2-norm of 
    the solution difference between two consecutive iterations.

    Parameters
    ----------
    p0 : numpy.ndarray
        The initial solution as a 2D array of floats
    b : numpy.ndarray
        The forcing term as a 2D array of floats
    dx : float
        Grid spacing in the x direction
    dy : float
        Grid spacing in the y direction
    maxiter : integer, optional
        Maximum number of iterations to perform, default: 20000
    rtol : float, optional
        Relative tolerance for convergence, default: 1e-6

    Returns
    -------
    p : numpy.ndarray
        The solution after relaxation as a 2D array of floats
    ite : integer
        The number of iterations performed
    conv : list
        The convergence history as a list of floats
    """
    
    def A(p):
        # apply the Laplacian operator to p
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ad = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    
    # initial residual
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    
    # initial search direction to be the residual
    d = r.copy()
    
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        
        # Laplacian of the search direction
        Ad[1:-1, 1:-1] = A(d)
        
        # step size
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
        
        # update the solution
        p = pk + alpha * d
        
        # update the residual
        r = rk - alpha * Ad
        
        # update the search direction
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        d = r + beta * d
        
        # relative L2-norm of the difference
        diff = l2_norm(p, pk)
        
        conv.append(diff)
        ite += 1
    
    return p, ite, conv

In [7]:
p, ites, conv_cg = poisson_2d_conjugate_gradient(p0, b, dx, dy, 
                                                 maxiter=20000, rtol=1e-10)

print('Method of conjugate gradients: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_cg[-1]))

Method of conjugate gradients: 2 iterations to reach a relative difference of 1.2982770796281907e-16


# Diffcult Poisson Problem
$$
\begin{equation}
b = \sin\left(\frac{\pi x}{L_x}\right) \cos\left(\frac{\pi y}{L_y}\right) + \sin\left(\frac{6\pi x}{L_x}\right) \cos\left(\frac{6\pi y}{L_y}\right)
\end{equation}
$$

In [8]:
# source term for the Poission system
b = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) +
     numpy.sin(6.0 * numpy.pi * X / Lx) *
     numpy.sin(6.0 * numpy.pi * Y / Ly))

In [9]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b, dx, dy, maxiter=maxiter, 
                                  rtol=rtol)
print('Jacobi relaxation: {} iterations'.format(ites))

p, ites, conv = poisson_2d_steepest_descent(p0, b, dx, dy, 
                                            maxiter=maxiter,rtol=rtol)
print('Method of steepest descent: {} iterations'.format(ites))

p, ites, conv = poisson_2d_conjugate_gradient(p0, b, dx, dy, 
                                              maxiter=maxiter, rtol=rtol)
print('Method of conjugate gradients: {} iterations'.format(ites))

Jacobi relaxation: 31226 iterations
Method of steepest descent: 27059 iterations
Method of conjugate gradients: 3 iterations
